In [1]:
import pandas as pd 
import numpy as np 
import os 
import requests
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [2]:
import requests
import pprint

teams = [
    
    "BOS",  # Boston Bruins
    "ANA",  # Anaheim Ducks
    "BUF",  # Buffalo Sabres
    "CAR",  # Carolina Hurricanes
    "CBJ",  # Columbus Blue Jackets
    "CGY",  # Calgary Flames
    "CHI",  # Chicago Blackhawks
    "COL",  # Colorado Avalanche
    "DAL",  # Dallas Stars
    "DET",  # Detroit Red Wings
                                     "EDM",  # Edmonton Oilers
    "FLA",  # Florida Panthers
    "LAK",  # Los Angeles Kings
    "MIN",  # Minnesota Wild
    "MTL",  # Montreal Canadiens
    "NJD",  # New Jersey Devils
    "NSH",  # Nashville Predators
    "NYI",  # New York Islanders
                                     "NYR",  # New York Rangers
    "OTT",  # Ottawa Senators
    "PHI",  # Philadelphia Flyers
    "PIT",  # Pittsburgh Penguins
    "SEA",  # Seattle Kraken
    "SJS",  # San Jose Sharks
    "STL",  # St. Louis Blues
    "TBL",  # Tampa Bay Lightning
    "TOR",  # Toronto Maple Leafs
    "UTA",  # Utah Hockey Club
                                    "VAN",  # Vancouver Canucks
    "VGK",  # Vegas Golden Knights
    "WPG",  # Winnipeg Jets
    "WSH",   # Washington Capitals
    "ARI",   # Arizona Coyotes
    "ATL",   # Atlanta Thrashers 
    "HFD",   # Hartford
    "PHX",   # Pheonix
    "QUE",   # Quebec
    "WIN",   # Winnepeg Jets - Old
     ]

seasons = [

    "20082009", "20092010",
    "20102011", "20112012", "20122013", "20132014", "20142015",
    "20152016", "20162017", "20172018", "20182019", "20192020",
    "20202021", "20212022", "20222023", "20232024", "20242025"
]

# Function to get player data from the NHL API
def get_player_data(team, season):
    url = f"https://api-web.nhle.com/v1/roster/{team}/{season}"
    response = requests.get(url)

    # Check if the response is valid JSON
    try:
        return response.json()
    except requests.JSONDecodeError:
        print(f"Error decoding JSON for team {team} in season {season}")
        return {}

# Initialize a dictionary to store player IDs
player_ids = {}

# Loop through each season in the seasons list
for season in seasons:
    player_ids[season] = {}

    # Loop through each team
    for team in teams:
        try:
            # Get the player data for the team and season
            player_data = get_player_data(team, season)
        
            # Initialize dictionaries for defencemen, forwards, and goalies
            player_ids[season][team] = {
                'defencemen': [],
                #'forwards': [],
                #'goalies': []
            }
        
            # Extract player IDs from defencemen
            if 'defensemen' in player_data:
                for player in player_data['defensemen']:
                    player_id = player['id']
                    player_ids[season][team]['defencemen'].append(player_id)
        
            # # Extract player IDs from forwards
            # if 'forwards' in player_data:
            #     for player in player_data['forwards']:
            #         player_id = player['id']
            #         player_ids[season][team]['forwards'].append(player_id)
        
            # # Extract player IDs from goalies
            # if 'goalies' in player_data:
            #     for player in player_data['goalies']:
            #         player_id = player['id']
            #         player_ids[season][team]['goalies'].append(player_id)
    
        except KeyError:
            print(f"Team {team} not found for season {season}. Skipping...")
            continue

#Print the player IDs
pprint.pprint(player_ids)

Error decoding JSON for team SEA in season 20082009
Error decoding JSON for team UTA in season 20082009
Error decoding JSON for team VGK in season 20082009
Error decoding JSON for team WPG in season 20082009
Error decoding JSON for team ARI in season 20082009
Error decoding JSON for team HFD in season 20082009
Error decoding JSON for team QUE in season 20082009
Error decoding JSON for team WIN in season 20082009
Error decoding JSON for team SEA in season 20092010
Error decoding JSON for team UTA in season 20092010
Error decoding JSON for team VGK in season 20092010
Error decoding JSON for team WPG in season 20092010
Error decoding JSON for team ARI in season 20092010
Error decoding JSON for team HFD in season 20092010
Error decoding JSON for team QUE in season 20092010
Error decoding JSON for team WIN in season 20092010
Error decoding JSON for team SEA in season 20102011
Error decoding JSON for team UTA in season 20102011
Error decoding JSON for team VGK in season 20102011
Error decodi

In [3]:
# Function to get game logs for a specific season
def get_game_log(player_id, season):
    url = f"https://api-web.nhle.com/v1/player/{player_id}/game-log/{season}/2"
    response = requests.get(url)
    return response.json()

# Check if the response is valid JSON
    try:
        return response.json()
    except requests.JSONDecodeError:
        print(f"Error decoding JSON for player {player_id} in season {season}")
        return {}

In [4]:
import pandas as pd

# Function to aggregate game logs for a specific player_id
def aggregate_game_logs(game_logs):
    aggregated_data = {}
    
    for game in game_logs:
        for key, value in game.items():
            if key not in aggregated_data:
                aggregated_data[key] = value
            else:
                # If the key already exists, aggregate the data (e.g., sum the values)
                if isinstance(value, (int, float)):
                    aggregated_data[key] += value
                elif isinstance(value, list):
                    aggregated_data[key].extend(value)
                # Handle other data types as needed
    
    return aggregated_data



# Initialize a dictionary to store DataFrames for each player_id
player_dfs = {}

# Loop through the selected player_ids
for season, teams in player_ids.items():
    for team, players in teams.items():
        for player_type, player_ids in players.items():
            for player_id in player_ids:
                game_log = get_game_log(player_id, season)
                
                if game_log and 'gameLog' in game_log:
                    game_log_data = game_log['gameLog']

                    seasons_list = []
                    
                    for stat in game_log_data:
                        stats_info = {
                              
                        'gameDate': stat.get('gameDate'),
                        'homeRoadFlag': stat.get('homeRoadFlag'),	
                        'goals': stat.get('goals'),	
                        'assists': stat.get('assists'),	
                        'points': stat.get('points'),	
                        'plusMinus': stat.get('plusMinus'),	
                        'powerPlayGoals': stat.get('powerPlayGoals'),	
                        'powerPlayPoints': stat.get('powerPlayGoals'),	
                        'gameWinningGoals': stat.get('gameWinningGoals'),	
                        'otGoals': stat.get('otGoals'),	
                        'shots': stat.get('shots'),	
                        'shifts': stat.get('shifts'),	
                        'opponentAbbrev': stat.get('opponentAbbrev'),	
                        'pim': stat.get('pim'),	
                        'toi': stat.get('toi'),    
                        
                        }
                        seasons_list.append(stats_info)
                    
                    # Create df from the flattened list of data
                    player_df = pd.DataFrame(seasons_list)

                    #  # Set gameDate as the index
                    # player_df.set_index('gameDate', inplace=True)

                    # Add the DataFrame to the player_dfs dictionary
                    player_dfs[player_id] = player_df

  
                   # Save the DataFrame to a CSV file in the specified folder
                    csv_filename = os.path.join('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence', f'player_{player_id}.csv')
                    player_df.to_csv(csv_filename, index=False)
                    print(f"DataFrame for player_id {player_id} saved to {csv_filename}")
                else:
                    print(f"No valid data found for player_id {player_id} in season {season}")




DataFrame for player_id 8470187 saved to /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8470187.csv
DataFrame for player_id 8465009 saved to /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8465009.csv
DataFrame for player_id 8466333 saved to /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8466333.csv
DataFrame for player_id 8460661 saved to /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8460661.csv
DataFrame for player_id 8471436 saved to /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8471436.csv
DataFrame for player_id 8471696 saved to /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8471696.csv
DataFrame for player_id 8468927 saved to /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8468927.csv
DataFrame for player_id 8470614 saved to /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8470

In [13]:
# Function to load all CSV files into a dictionary of DataFrames
def load_all_csv_files(directory):
    csv_files = glob.glob(os.path.join(directory, '/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_*.csv'))
    dataframes = {}
    
    for csv_file in csv_files:
        player_id = os.path.basename(csv_file).split('_')[1].split('.')[0]
        try:
            df = pd.read_csv(csv_file)
            if df.empty:
                print(f"Empty CSV file: {csv_file}. Skipping...")
                continue
            dataframes[player_id] = df
        except pd.errors.EmptyDataError:
            print(f"No columns to parse from file: {csv_file}. Skipping...")
            continue
    
    return dataframes

# Specify the directory where the CSV files are saved
directory = '.'  # Current directory

# Load all CSV files into a dictionary of DataFrames
player_dfs = load_all_csv_files(directory)

No columns to parse from file: /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8480222.csv. Skipping...
No columns to parse from file: /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8480828.csv. Skipping...
No columns to parse from file: /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8482187.csv. Skipping...
No columns to parse from file: /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8478846.csv. Skipping...
No columns to parse from file: /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8477433.csv. Skipping...
No columns to parse from file: /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8481614.csv. Skipping...
No columns to parse from file: /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/player_8482641.csv. Skipping...
No columns to parse from file: /Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapidefence/

In [14]:
# Function to create a new DataFrame grouped by homeRoadFlag
def create_grouped_df(df):
    grouped_df = df.groupby('homeRoadFlag').agg({
        'gameDate': 'count',
        'goals': 'sum',
        'assists': 'sum',
        'points': 'sum',
        'plusMinus': 'sum',
        'powerPlayGoals': 'sum',
        'powerPlayPoints': 'sum',
        'gameWinningGoals': 'sum',
        'otGoals': 'sum',
        'shots': 'sum',
        'shifts': 'sum',
        'pim': 'sum',
        
    }).rename(columns={'gameDate': 'gameCount'})
    # Sum the grouped statistics to create a single row DataFrame
    single_row_df = grouped_df.sum().to_frame().T
    
    return single_row_df

In [15]:
# Loop through each DataFrame and create a grouped DataFrame
grouped_dfs = {}
for player_id, df in player_dfs.items():
    try:
        # Check if 'gameDate' column has values
        if df['gameDate'].empty:
            print(f"No gameDate values found for player_id {player_id}. Skipping...")
            continue

        # Check if 'homeRoadFlag' column exists
        if 'homeRoadFlag' in df.columns:
            grouped_df = create_grouped_df(df)
            grouped_dfs[player_id] = grouped_df
        else:
            print(f"'homeRoadFlag' column not found for player_id {player_id}. Skipping...")
    except Exception as e:
        print(f"An error occurred for player_id {player_id}: {e}")

grouped_dfs

{'8470672':    gameCount  goals  assists  points  plusMinus  powerPlayGoals  \
 0         42      2        4       6         -4               0   
 
    powerPlayPoints  gameWinningGoals  otGoals  shots  shifts  pim  
 0                0                 0        0     26    1093   60  ,
 '8471236':    gameCount  goals  assists  points  plusMinus  powerPlayGoals  \
 0         60      7        7      14        -13               1   
 
    powerPlayPoints  gameWinningGoals  otGoals  shots  shifts  pim  
 0                1                 1        0     67    1429   36  ,
 '8467332':    gameCount  goals  assists  points  plusMinus  powerPlayGoals  \
 0         11      0        2       2         -2               0   
 
    powerPlayPoints  gameWinningGoals  otGoals  shots  shifts  pim  
 0                0                 0        0      6     218    6  ,
 '8471222':    gameCount  goals  assists  points  plusMinus  powerPlayGoals  \
 0         22      0        0       0         -7         

In [16]:
# Concatenate all DataFrames into a master DataFrame
master_df = pd.concat(grouped_dfs.values(), keys=grouped_dfs.keys(), names=['player_id', 'index'])

# Reset the index to have 'player_id' as a column
master_df.reset_index(level='index', drop=True, inplace=True)
master_df.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/master_copies/defence.csv')